In [2]:
import os
import torch
import torch.nn.functional as F
from tqdm import tqdm
from torch_geometric.loader import NeighborLoader
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torch_geometric.nn import MessagePassing, SAGEConv
from ogb.nodeproppred import Evaluator, PygNodePropPredDataset
import yfinance as yf
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from torch_geometric.data import Data
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score
import seaborn as sns
import matplotlib.pyplot as plt
from early_stopping import EarlyStopping
import random
from yf_dataset import getInput

seed = 42
os.environ['PYTHONHASHSEED'] = str(seed)
random.seed(seed)
np.random.seed(seed)
torch.manual_seed(seed)
torch.cuda.manual_seed(seed)
torch.cuda.manual_seed_all(seed)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False

In [3]:
train_idx, eval_idx, test_idx, weight, yfdata, gnnInputData = getInput(False, False, withMacdSignal=True, macdParamOptimize=False)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = torch.load(r'C:\Users\tony\Desktop\GNN_test\result\CrossEntrophyLoss\CrossEntrophyLoss with MACD Signal startLr=1.869129e-04 numNeighbors=18\best_model.pt')

[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed
[*********************100%%**********************]  1 of 1 completed


c:\Users\tony\Desktop\GNN_test\yf_dataset.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yfdata['Signal'].iloc[i] = 0
c:\Users\tony\Desktop\GNN_test\yf_dataset.py:97: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yfdata['Signal'].iloc[i] = 2
c:\Users\tony\Desktop\GNN_test\yf_dataset.py:95: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  yfdata['Signal'].iloc[i] = 0
c:\Users\tony\Desktop\GNN_test\yf_dataset.py:97: SettingWithCopyWar

If there is NAN in data?  False
Index(['2011-01-31 00:00:00 ~ 2011-02-09 00:00:00',
       '2011-02-01 00:00:00 ~ 2011-02-10 00:00:00',
       '2011-02-02 00:00:00 ~ 2011-02-11 00:00:00',
       '2011-02-03 00:00:00 ~ 2011-02-14 00:00:00',
       '2011-02-04 00:00:00 ~ 2011-02-15 00:00:00',
       '2011-02-07 00:00:00 ~ 2011-02-16 00:00:00',
       '2011-02-08 00:00:00 ~ 2011-02-17 00:00:00',
       '2011-02-09 00:00:00 ~ 2011-02-18 00:00:00',
       '2011-02-10 00:00:00 ~ 2011-02-22 00:00:00',
       '2011-02-11 00:00:00 ~ 2011-02-23 00:00:00',
       ...
       '2019-08-08 00:00:00 ~ 2019-08-19 00:00:00',
       '2019-08-09 00:00:00 ~ 2019-08-20 00:00:00',
       '2019-08-12 00:00:00 ~ 2019-08-21 00:00:00',
       '2019-08-13 00:00:00 ~ 2019-08-22 00:00:00',
       '2019-08-14 00:00:00 ~ 2019-08-23 00:00:00',
       '2019-08-15 00:00:00 ~ 2019-08-26 00:00:00',
       '2019-08-16 00:00:00 ~ 2019-08-27 00:00:00',
       '2019-08-19 00:00:00 ~ 2019-08-28 00:00:00',
       '2019-08-20 00

c:\Users\tony\Desktop\GNN_test\yf_dataset.py:193: UserWarning: Creating a tensor from a list of numpy.ndarrays is extremely slow. Please consider converting the list to a single numpy.ndarray with numpy.array() before converting to a tensor. (Triggered internally at ..\torch\csrc\utils\tensor_new.cpp:248.)
  edge_index = torch.tensor([


feature_matrix_shape =  torch.Size([2154, 42])
edge size=  torch.Size([2, 34423])
y =  torch.Size([2154, 1])
找到日期 '2013-01-18 00:00:00 ~ 2013-01-30 00:00:00'，index= 495
找到日期 '2017-05-12 00:00:00 ~ 2017-05-23 00:00:00'，index= 1581
找到日期 '2018-03-16 00:00:00 ~ 2018-03-27 00:00:00'，index= 1793
找到日期 '2019-08-20 00:00:00 ~ 2019-08-29 00:00:00'，index= 2152
上漲(0)的数量： 534
持有(1)的数量： 1203
下跌(2)的数量： 417
weight:  [4.033707865093001, 1.7905236907581836, 5.165467625775408]


In [4]:
# mlGAT_period = torch.tensor(np.arange(1361,2671+1),dtype=torch.long)
total_loader = NeighborLoader(gnnInputData, input_nodes=test_idx, num_neighbors=[-1],
                                shuffle=False,
                                num_workers=os.cpu_count() - 2)

In [5]:
model.eval()
model.to(device)
out, var = model.inference(total_loader, device)
# y_pred = out.argmax(dim=-1)
y_pred = torch.exp(out) 

In [6]:
start_date = "2011-01-30"
end_date = "2019-09-01"
spy = yf.download('SPY', start=start_date, end=end_date)

[*********************100%%**********************]  1 of 1 completed


In [14]:
label_data = spy['2018-03-28':]
label_data = label_data[:-1]
init_investment = 10000
investment = init_investment
sAndp_hold = 0
total = []
sell_count = 0
buy_count = 0
for index, pred in enumerate(y_pred):
    open = label_data.iloc[index]['Open']
    adj_close = label_data.iloc[index]['Adj Close']

    if ((pred.argmax(dim=-1) == 0) and (investment >= adj_close)):
        trade_volumn = int (investment / adj_close * pred[pred.argmax(dim=-1)])
        investment -= adj_close * trade_volumn
        sAndp_hold += trade_volumn
        buy_count += trade_volumn
    elif ((pred.argmax(dim=-1) == 2) and (sAndp_hold > 0)):
        trade_volumn = int (investment / adj_close * pred[pred.argmax(dim=-1)])
        investment += adj_close * trade_volumn
        sAndp_hold -= trade_volumn
        sell_count += trade_volumn

    total.append(investment + (sAndp_hold * adj_close))
          
total

[10000.0,
 10000.0,
 9917.734619140625,
 9926.696212768555,
 9974.680953979492,
 10010.453811645508,
 9908.637573242188,
 9930.650680541992,
 10043.348892211914,
 9998.052810668945,
 10068.766860961914,
 10043.348892211914,
 10114.38703918457,
 10207.588455200195,
 10214.104995727539,
 10165.224624633789,
 10090.924087524414,
 10089.623306274414,
 9972.635391235352,
 9992.642349243164,
 10065.418075561523,
 10073.112899780273,
 10010.019180297852,
 10024.485885620117,
 9969.70280456543,
 9951.852493286133,
 10056.493179321289,
 10086.636413574219,
 10086.636413574219,
 10168.375534057617,
 10257.338333129883,
 10286.640701293945,
 10291.228271484375,
 10224.859359741211,
 10265.104293823242,
 10256.986038208008,
 10232.976425170898,
 10304.994552612305,
 10277.478073120117,
 10304.63444519043,
 10284.359176635742,
 10260.825241088867,
 10147.492721557617,
 10277.478073120117,
 10217.013595581055,
 10313.324630737305,
 10360.39616394043,
 10367.63737487793,
 10450.914352416992,
 10449.8

In [15]:
import plotly.graph_objects as go
fig = go.Figure()
fig.add_trace(go.Scatter(x=label_data.index, y=list(map(lambda x:x/init_investment, total)), name='Return Percentage'))

fig.update_layout(title="S&P500")
fig.show()
# daily_return = list(map(lambda idx_val: (idx_val[1] - total[idx_val[0] - 1]) / total[idx_val[0] - 1] if idx_val[0] > 0 else 0, enumerate(total)))
print('average_daily_return = ',((total[-1] / init_investment) - 1) / len(total))

average_daily_return =  0.00035764903969087296
